In [1]:
def ambil_best(grid_search,n):
    cv_r=pd.DataFrame(grid_search.cv_results_)
    cv_r_v=cv_r.sort_values(['rank_test_score'])
    cv_r_v=cv_r_v.head(n)['params'].values

    for wi in range(len(cv_r_v)):

        for i in list(cv_r_v[wi]):

            cv_r_v[wi][i]=[cv_r_v[wi][i]]


    return cv_r_v

In [2]:




def input_mlflow(cv_r_v,increment_number,X_train, y_train,pipeline,run_name,experiment_id):
    

    for wi in range(len(cv_r_v)):
        
        #====================skema===================
        cols_spec = []
        data_map = {
                'int64': 'integer',
                'float64': 'double',
                'bool': 'boolean',
                'str': 'string',
                "date": 'datetime'
            }

        for name, dtype in X_train.dtypes.to_dict().items():
            cols_spec.append(ColSpec(name=name, type=data_map[str(dtype)]))
        input_schema = Schema(inputs=cols_spec)
        output_schema = Schema([ColSpec(name="label", type="double")])
        #parameter = ParamSpec(name="model_name", dtype="string", default="model1")
        #param_schema = ParamSchema(params=[parameter])
        model_signature = ModelSignature(inputs=input_schema, outputs=output_schema)#, params=param_schema)
        #print("MODEL SIGNATURE")
        #print(model_signature.to_dict())

        model_signature = infer_signature(X_train, y_train)#, params={"model_name": "model1"})
        #print("MODEL SIGNATURE")
        #print(model_signature.to_dict())

        
        
        
        
        #====================buat run baru===================
        run_name_with_increment = f"{run_name}__{increment_number}"
            # Membuka run MLflow
        with mlflow.start_run(run_name=run_name_with_increment, experiment_id=experiment_id) as run:
            # Mendapatkan run_id
            run_id = mlflow.active_run().info.run_id
        increment_number=increment_number+1
        print("run_id:", run_id)

        
        
        #====================buat parameter model dan metric===================

        # Definisikan grid parameter untuk dicari
        param_grid = cv_r_v[wi]
        
        #====================model===================

        # Inisialisasi GridSearchCV
        grid_search = GridSearchCV(pipeline, param_grid, cv=5,  scoring='neg_mean_squared_error')

        # Lakukan pencarian grid
        grid_search.fit(X_train, y_train)
        
            
        #====================parameter===================
        pipe=grid_search.best_estimator_
        
        
        first_step_name = list(pipe.named_steps.keys())[0:len(pipe)]
        

        for i in range(len(pipe)):
            # Mendapatkan parameter dari langkah 'sca'
            nama=first_step_name[i]
            sca_params = pipe.get_params()[nama]

                # Membuka run MLflow
            with mlflow.start_run( experiment_id=experiment_id,run_id=run_id) as run:

                # Log parameter secara otomatis menggunakan loop
                for param_name, param_value in sca_params.get_params().items():
                    param_name=nama+'__'+param_name
                    #print(param_name,param_value)
                    mlflow.log_param(param_name, param_value)
                    
                    
                #====================metric===================
                #matric
                hasil_test=grid_search.predict(X_test)
                mae=mean_absolute_error(y_test, hasil_test)
                mse=mean_squared_error(y_test, hasil_test)
                rmse=np.sqrt(mse)
                r2=r2_score(y_test, hasil_test)
                #print(mse)
                

                # log model 
                mlflow.sklearn.log_model(sk_model=grid_search, artifact_path="grid_search__"+str(increment_number-1),signature=model_signature)

                metrics = {
                    'mean_test_score':pd.DataFrame(grid_search.cv_results_)['mean_test_score'].values[0],
                    "mse": mse,
                    "mae": mae,
                    "rmse": rmse,
                    "r2": r2
                }

                mlflow.log_metrics(metrics)
                
                
        mlflow.end_run()
    print('selesai')

        
    return increment_number


            
        

In [39]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats



import seaborn as sns
import statsmodels.api as sm
import pandas as pd



from scipy.stats import shapiro,normaltest,kstest,jarque_bera
import pingouin as pg
from pingouin import kruskal


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import mlflow


from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score


import mlflow

from mlflow.models.signature import ModelSignature
from mlflow.models.signature import infer_signature
from mlflow.types.schema import Schema
from mlflow.types.schema import ParamSchema
from mlflow.types.schema import ParamSpec
from mlflow.types.schema import ColSpec
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import pandas as pd
from typing import Tuple


# import data

In [208]:
df=pd.read_csv('data/data_csv/data_bersih.csv')
df.head()


,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [209]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         199 non-null    float64
 1   radio      199 non-null    float64
 2   newspaper  199 non-null    float64
 3   sales      199 non-null    float64
dtypes: float64(4)
memory usage: 6.3 KB


In [210]:
df.describe()

,TV,radio,newspaper,sales
count,199.000000,199.000000,199.000000,199.000000
mean,147.440704,23.196985,30.134673,14.030151
std,85.885409,14.853898,21.008552,5.229491
min,0.700000,0.000000,0.300000,1.600000
25%,74.900000,9.950000,12.700000,10.350000
50%,149.800000,22.500000,25.600000,12.900000
75%,219.150000,36.400000,44.700000,17.400000
max,296.400000,49.600000,100.900000,27.000000


# split data

In [211]:
X = df.drop(['sales'],axis=1)
y = df['sales']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=101)

In [212]:
len(X_train), len(X_test)

(179, 20)

In [213]:
from sklearn.feature_selection import mutual_info_regression
mutual_info=mutual_info_regression(X_train, y_train)

mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns
mutual_info.sort_values(ascending=False)

TV           0.630882
radio        0.445911
newspaper    0.000000
dtype: float64

# tandai newspaper    0.000000

# bangun pipeline

In [214]:
from sklearn import set_config
set_config(display='diagram')

In [215]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly=PolynomialFeatures()
lin=LinearRegression()
scatler=StandardScaler()

opera=[('StandardScaler',scatler),('PolynomialFeatures',poly),('LinearRegression',lin)]
pipeline=Pipeline(opera)



# Definisikan grid parameter untuk dicari
param_grid = {
    'PolynomialFeatures__degree': [2, 3, 4,5,6,7],        # Derajat polinomial yang akan dicoba
    
    'LinearRegression__fit_intercept': [True, False]# Pilihan untuk normalisasi regresi
}


from sklearn.model_selection import GridSearchCV
# Inisialisasi GridSearchCV


# Inisialisasi GridSearchCV dengan menggunakan MAE sebagai skor evaluasi
grid_search = GridSearchCV(pipeline, param_grid, cv=5,  scoring='neg_mean_squared_error')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('PolynomialFeatures',
                                        PolynomialFeatures()),
                                       ('LinearRegression',
                                        LinearRegression())]),
             param_grid={'LinearRegression__fit_intercept': [True, False],
                         'PolynomialFeatures__degree': [2, 3, 4, 5, 6, 7]},
             scoring='neg_mean_squared_error')

In [216]:

df_1 = pd.DataFrame(grid_search.cv_results_)
df_1.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_1 = df_1.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_1.head(20)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LinearRegression__fit_intercept,param_PolynomialFeatures__degree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,0.004686,0.007310,0.002314,0.004628,False,3,"{'LinearRegression__fit_intercept': False, 'Po...",-0.779523,-0.662875,-0.126662,-0.166336,-0.197063,-0.386492,0.276668,1
1,0.008185,0.004588,0.005390,0.003412,True,3,"{'LinearRegression__fit_intercept': True, 'Pol...",-0.779523,-0.662875,-0.126662,-0.166336,-0.197063,-0.386492,0.276668,2
8,0.007947,0.007245,0.000000,0.000000,False,4,"{'LinearRegression__fit_intercept': False, 'Po...",-0.656132,-0.608026,-0.204625,-0.214348,-0.279519,-0.392530,0.197865,3
0,0.009202,0.002804,0.002658,0.001570,True,2,"{'LinearRegression__fit_intercept': True, 'Pol...",-1.100718,-0.207837,-0.315704,-0.193970,-0.286509,-0.420948,0.342979,4
6,0.003535,0.005624,0.002771,0.005541,False,2,"{'LinearRegression__fit_intercept': False, 'Po...",-1.100718,-0.207837,-0.315704,-0.193970,-0.286509,-0.420948,0.342979,5
9,0.014245,0.003947,0.000696,0.001391,False,5,"{'LinearRegression__fit_intercept': False, 'Po...",-0.307708,-3.524462,-0.122986,-0.223158,-0.213656,-0.878394,1.324327,6
2,0.011293,0.007498,0.005857,0.006223,True,4,"{'LinearRegression__fit_intercept': True, 'Pol...",-0.650619,-56.244333,-0.204625,-0.214348,-0.279519,-11.518689,22.363422,7
3,0.015344,0.002761,0.003006,0.002512,True,5,"{'LinearRegression__fit_intercept': True, 'Pol...",-0.378271,-61.926897,-1.279254,-1.145503,-0.686918,-13.083368,24.423888,8
10,0.026636,0.001614,0.001055,0.001151,False,6,"{'LinearRegression__fit_intercept': False, 'Po...",-0.902142,-1631.565254,-0.469634,-0.414732,-0.240617,-326.718476,652.423425,9
4,0.026652,0.005320,0.003267,0.001986,True,6,"{'LinearRegression__fit_intercept': True, 'Pol...",-3.409551,-2610.186580,-0.686341,-0.761582,-0.232301,-523.055271,1043.566254,10


# coba drop newspaper

In [217]:
X = df.drop(['sales','newspaper'],axis=1)
y = df['sales']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=101)
len(X_train), len(X_test)

(179, 20)

In [218]:
# Inisialisasi GridSearchCV dengan menggunakan MAE sebagai skor evaluasi
grid_search = GridSearchCV(pipeline, param_grid, cv=5,  scoring='neg_mean_squared_error')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('PolynomialFeatures',
                                        PolynomialFeatures()),
                                       ('LinearRegression',
                                        LinearRegression())]),
             param_grid={'LinearRegression__fit_intercept': [True, False],
                         'PolynomialFeatures__degree': [2, 3, 4, 5, 6, 7]},
             scoring='neg_mean_squared_error')

In [219]:

df_2 = pd.DataFrame(grid_search.cv_results_)
df_2.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_2 = df_2.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_2.head(20)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LinearRegression__fit_intercept,param_PolynomialFeatures__degree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.007565,0.007006,0.003125,0.006249,False,6,"{'LinearRegression__fit_intercept': False, 'Po...",-0.545429,-0.090906,-0.130978,-0.126439,-0.197246,-0.218200,0.167182,1
8,0.006237,0.007639,0.000000,0.000000,False,4,"{'LinearRegression__fit_intercept': False, 'Po...",-0.676758,-0.153785,-0.147109,-0.104928,-0.139138,-0.244343,0.216860,2
2,0.002947,0.005893,0.003500,0.006999,True,4,"{'LinearRegression__fit_intercept': True, 'Pol...",-0.676758,-0.153785,-0.147109,-0.104928,-0.139138,-0.244343,0.216860,3
9,0.003092,0.006185,0.003164,0.006327,False,5,"{'LinearRegression__fit_intercept': False, 'Po...",-0.624860,-0.107698,-0.153466,-0.111772,-0.225247,-0.244609,0.194763,4
3,0.006219,0.006492,0.003562,0.006112,True,5,"{'LinearRegression__fit_intercept': True, 'Pol...",-0.624860,-0.107698,-0.153466,-0.111772,-0.225247,-0.244609,0.194763,5
1,0.007164,0.005585,0.003313,0.005887,True,3,"{'LinearRegression__fit_intercept': True, 'Pol...",-0.827129,-0.124151,-0.145622,-0.122261,-0.134401,-0.270713,0.278333,6
7,0.003126,0.006251,0.003125,0.006250,False,3,"{'LinearRegression__fit_intercept': False, 'Po...",-0.827129,-0.124151,-0.145622,-0.122261,-0.134401,-0.270713,0.278333,7
11,0.005038,0.006461,0.000000,0.000000,False,7,"{'LinearRegression__fit_intercept': False, 'Po...",-0.714735,-0.106329,-0.178220,-0.138407,-0.220179,-0.271574,0.224850,8
4,0.006364,0.006424,0.002514,0.005029,True,6,"{'LinearRegression__fit_intercept': True, 'Pol...",-0.470531,-0.168575,-0.188172,-0.283926,-0.331324,-0.288505,0.109030,9
5,0.006257,0.007663,0.003222,0.006444,True,7,"{'LinearRegression__fit_intercept': True, 'Pol...",-1.262111,-0.106411,-0.259148,-0.189283,-0.248678,-0.413126,0.427950,10


In [220]:
df_gabungan=pd.concat([df_1.head(7),df_2.head(7)])
df_gabungan.drop_duplicates(subset=['mean_test_score', 'std_test_score'], inplace=True)
df_gabungan.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LinearRegression__fit_intercept,param_PolynomialFeatures__degree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.007565,0.007006,0.003125,0.006249,False,6,"{'LinearRegression__fit_intercept': False, 'Po...",-0.545429,-0.090906,-0.130978,-0.126439,-0.197246,-0.218200,0.167182,1
8,0.006237,0.007639,0.000000,0.000000,False,4,"{'LinearRegression__fit_intercept': False, 'Po...",-0.676758,-0.153785,-0.147109,-0.104928,-0.139138,-0.244343,0.216860,2
2,0.002947,0.005893,0.003500,0.006999,True,4,"{'LinearRegression__fit_intercept': True, 'Pol...",-0.676758,-0.153785,-0.147109,-0.104928,-0.139138,-0.244343,0.216860,3
9,0.003092,0.006185,0.003164,0.006327,False,5,"{'LinearRegression__fit_intercept': False, 'Po...",-0.624860,-0.107698,-0.153466,-0.111772,-0.225247,-0.244609,0.194763,4
3,0.006219,0.006492,0.003562,0.006112,True,5,"{'LinearRegression__fit_intercept': True, 'Pol...",-0.624860,-0.107698,-0.153466,-0.111772,-0.225247,-0.244609,0.194763,5
1,0.007164,0.005585,0.003313,0.005887,True,3,"{'LinearRegression__fit_intercept': True, 'Pol...",-0.827129,-0.124151,-0.145622,-0.122261,-0.134401,-0.270713,0.278333,6
7,0.003126,0.006251,0.003125,0.006250,False,3,"{'LinearRegression__fit_intercept': False, 'Po...",-0.827129,-0.124151,-0.145622,-0.122261,-0.134401,-0.270713,0.278333,7
7,0.004686,0.007310,0.002314,0.004628,False,3,"{'LinearRegression__fit_intercept': False, 'Po...",-0.779523,-0.662875,-0.126662,-0.166336,-0.197063,-0.386492,0.276668,1
1,0.008185,0.004588,0.005390,0.003412,True,3,"{'LinearRegression__fit_intercept': True, 'Pol...",-0.779523,-0.662875,-0.126662,-0.166336,-0.197063,-0.386492,0.276668,2
8,0.007947,0.007245,0.000000,0.000000,False,4,"{'LinearRegression__fit_intercept': False, 'Po...",-0.656132,-0.608026,-0.204625,-0.214348,-0.279519,-0.392530,0.197865,3


# ambil data paremeterterbaik

In [221]:
def ambil_best(grid_search,n):
    cv_r=grid_search.copy()
    cv_r_v=cv_r.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
    cv_r_v=cv_r_v.head(n)['params'].values

    for wi in range(len(cv_r_v)):

        for i in list(cv_r_v[wi]):

            cv_r_v[wi][i]=[cv_r_v[wi][i]]


    return cv_r_v
inis=ambil_best(df_gabungan,5)
inis

array([{'LinearRegression__fit_intercept': [False], 'PolynomialFeatures__degree': [6]},
       {'LinearRegression__fit_intercept': [False], 'PolynomialFeatures__degree': [4]},
       {'LinearRegression__fit_intercept': [True], 'PolynomialFeatures__degree': [4]},
       {'LinearRegression__fit_intercept': [False], 'PolynomialFeatures__degree': [5]},
       {'LinearRegression__fit_intercept': [True], 'PolynomialFeatures__degree': [5]}],
      dtype=object)

# buat experiment

In [222]:
if __name__ == "__main__":

    experiment_id = mlflow.create_experiment(
        name="project_poly_new",
        tags={"env": "dev", "version": "1.0.0"},
    )

    print(experiment_id)
    
run_name = "log___"
increment_number = 1
mlflow.end_run()

increment_number=input_mlflow(inis,increment_number,X_train, y_train,pipeline,run_name,experiment_id)

675159362592057773
run_id: 445d792497f04f64af652889801d630d
run_id: a46027623bcf4d018aec2b659c25610f
run_id: c937e595063f47d6beafff1a964bfccf
run_id: f37e19b2b5df4a0cbc9965b0d7d65c34
run_id: 9a86738546b04512a5d48bb50a6b0363
selesai


# **make voting**

In [223]:
model1 = mlflow.sklearn.load_model('mlruns/675159362592057773/445d792497f04f64af652889801d630d/artifacts/grid_search__1')
model2 = mlflow.sklearn.load_model('mlruns/675159362592057773/a46027623bcf4d018aec2b659c25610f/artifacts/grid_search__2')

In [224]:
from sklearn.ensemble import VotingRegressor

voting_pipe = Pipeline([
    ('voting', VotingRegressor(estimators=[
        ('grid_search1', model1.best_estimator_),
         # Model terbaik dari grid search pertama
        ('grid_search2',model2.best_estimator_)   # Model terbaik dari grid search kedua
    ]))
])


In [225]:
voting_pipe.fit(X_train,y_train)


Pipeline(steps=[('voting',
                 VotingRegressor(estimators=[('grid_search1',
                                              Pipeline(steps=[('StandardScaler',
                                                               StandardScaler()),
                                                              ('PolynomialFeatures',
                                                               PolynomialFeatures(degree=6)),
                                                              ('LinearRegression',
                                                               LinearRegression(fit_intercept=False))])),
                                             ('grid_search2',
                                              Pipeline(steps=[('StandardScaler',
                                                               StandardScaler()),
                                                              ('PolynomialFeatures',
                                                               PolynomialFeatures(degree=4)),
                                                              ('LinearRegression',
                                                               LinearRegression(fit_intercept=False))]))]))])

In [226]:
mod1=model1.predict(X_test)
mod2=model2.predict(X_test)
vote=voting_pipe.predict(X_test)

In [227]:
y_test=y_test.reset_index(drop=True)

In [228]:
gabungan=pd.concat([y_test,pd.DataFrame(mod1),pd.DataFrame(mod2),pd.DataFrame(vote)],axis=1)
gabungan.columns=(['Actual','Predicted 1','Predicted 2','vote'])
gabungan=gabungan.sort_values(['Actual'])
gabungan=gabungan.reset_index(drop=True)
gabungan

,Actual,Predicted 1,Predicted 2,vote
0,5.5,5.385011,5.657039,5.521025
1,8.4,9.034948,8.974578,9.004763
2,8.7,8.121732,7.845081,7.983407
3,9.6,9.334687,9.534606,9.434647
4,10.1,10.397085,9.845846,10.121465
5,10.1,10.428649,9.907873,10.168261
6,10.3,10.258620,10.139266,10.198943
7,10.9,10.807901,10.434358,10.621129
8,10.9,10.730477,10.839812,10.785144
9,11.2,10.843322,10.614538,10.728930


# **plot plotly**

In [229]:



import plotly.graph_objs as go
x=gabungan.index.values
# Define traces for each line
trace_actual = go.Scatter(x=x, y=gabungan['Actual'].values, mode='lines', name='Actual',line=dict(color='red'))
trace_pred_1 = go.Scatter(x=x, y=gabungan['Predicted 1'].values, mode='lines', name='Predicted 1',line=dict(color='blue'))
trace_pred_2 = go.Scatter(x=x, y=gabungan['Predicted 2'].values, mode='lines', name='Predicted 2',line=dict(color='yellow'))
trace_pred_v = go.Scatter(x=x, y=gabungan['vote'].values, mode='lines', name='Vote',line=dict(color='green'))

# Create the plotly figure
fig = go.Figure()

# Add traces to the figure
fig.add_trace(trace_actual)
fig.add_trace(trace_pred_1)
fig.add_trace(trace_pred_2)
fig.add_trace(trace_pred_v)

# Update layout
fig.update_layout(
    title="Comparison of Actual and Predicted Values",
    xaxis_title="X Axis Title",
    yaxis_title="Y Axis Title",
    #xaxis_tickfont=dict(size=10)
)

# Show plot
fig.show()

# **save model**

In [233]:
import joblib

# Simpan pipeline ke dalam file menggunakan joblib
joblib.dump(voting_pipe, "data/data_model/voting_pipeline.joblib")


['data/data_model/voting_pipeline.joblib']